# Load all images.

In [10]:
import os
from collections import defaultdict
from PIL import Image, ImageDraw
import re
from pathlib import Path

In [11]:
base_directory = '../../../teamspace/uploads'
base_directory_predictions = 'qwen/inference-results/'
image_count = 0
images_filenames = []
txt_count = 0
txt_filenames = []

with os.scandir(base_directory) as entries:
    for entry in entries:
        if entry.is_file() and entry.name.lower().endswith('.jpg'):
            image_count += 1
            images_filenames.append(entry.path)
        elif entry.is_file() and entry.name.lower().endswith('.txt'):
            txt_count += 1
            txt_filenames.append(entry.path)

print(f"Images in total: {len(images_filenames)}")
print(f"Text files in total: {len(txt_filenames)}")

Images in total: 43658
Text files in total: 48862


## Download the model

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

torch.manual_seed(1234)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cuda", trust_remote_code=True).eval()
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:260

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Query the model.

In [5]:
def normalize_and_format_bounding_boxes(matches, image_width, image_height):
    normalized_boxes = []
    for i in range(0, len(matches), 2):
        x1, y1 = map(int, matches[i].strip('()').split(','))
        x2, y2 = map(int, matches[i+1].strip('()').split(','))
        
        center_x = (x1 + x2) / 2 / image_width
        center_y = (y1 + y2) / 2 / image_height
        width = (x2 - x1) / image_width
        height = (y2 - y1) / image_height
        
        normalized_boxes.append(f"{center_x:.10f} {center_y:.10f} {width:.10f} {height:.10f}")
    return normalized_boxes

In [6]:
def draw_bounding_boxes(image, matches):
    draw = ImageDraw.Draw(image)
    for i in range(0, len(matches), 2):
        x1, y1 = map(int, matches[i].strip('()').split(','))
        x2, y2 = map(int, matches[i+1].strip('()').split(','))
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)

In [8]:
"""
for file_name in images_filenames:

    image_path = f"../../../teamspace/uploads/{file_name}"
    
    query = tokenizer.from_list_format([
        {'image': file_name},
        {'text': 'Give me the bounding box of drones in the photo, if any exist.'},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)

    # Use regex to find all coordinates inside parentheses
    matches = re.findall(r'\(\d+,\d+\)', response)
    
    if matches and len(matches) % 2 == 0:
        # Open the image to get its dimensions
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Normalize the coordinates and format as required
        formatted_bounding_boxes = normalize_and_format_bounding_boxes(matches, image_width, image_height)

        # Join the formatted bounding boxes into a single string separated by newlines
        result = '\n'.join(formatted_bounding_boxes)
        
        output_file_path = os.path.join("qwen/inference-results/", Path(file_name).stem + '.txt')

        print(f"Response for {file_name}: {response}. Normalized coordinates result: {result}")
        with open(output_file_path, 'w') as file:
            file.write(result)
    else:
        print(f"Response for {file_name}: {response}. No valid bounding boxes found.")
"""

'\nfor file_name in images_filenames:\n\n    image_path = f"../../../teamspace/uploads/{file_name}"\n    \n    query = tokenizer.from_list_format([\n        {\'image\': file_name},\n        {\'text\': \'Give me the bounding box of drones in the photo, if any exist.\'},\n    ])\n    response, history = model.chat(tokenizer, query=query, history=None)\n\n    # Use regex to find all coordinates inside parentheses\n    matches = re.findall(r\'\\(\\d+,\\d+\\)\', response)\n    \n    if matches and len(matches) % 2 == 0:\n        # Open the image to get its dimensions\n        with Image.open(image_path) as img:\n            image_width, image_height = img.size\n\n        # Normalize the coordinates and format as required\n        formatted_bounding_boxes = normalize_and_format_bounding_boxes(matches, image_width, image_height)\n\n        # Join the formatted bounding boxes into a single string separated by newlines\n        result = \'\n\'.join(formatted_bounding_boxes)\n        \n        o

In [8]:
image_prediction = 0
image_no_prediction = 0

for file_name in images_filenames:

    image_path = f"../../../teamspace/uploads/{file_name}"
    
    query = tokenizer.from_list_format([
        {'image': file_name},
        {'text': 'Give me the bounding box of drones in the photo, if any exist.'},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)

    # Use regex to find all coordinates inside parentheses
    matches = re.findall(r'\(\d+,\d+\)', response)
    
    if matches and len(matches) % 2 == 0:
        # Open the image to get its dimensions
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Normalize the coordinates and format as required
        formatted_bounding_boxes = normalize_and_format_bounding_boxes(matches, image_width, image_height)

        print(formatted_bounding_boxes)

        print(matches)
        #draw_bounding_boxes(img, matches)

        # Join the formatted bounding boxes into a single string separated by newlines
        result = '\n'.join(formatted_bounding_boxes)

        #img_with_boxes_path = os.path.join("qwen/inference-results/", Path(file_name).stem + '.jpg')
        #img.save(img_with_boxes_path)
        #image_prediction += 1
    else:
        # No valid bounding boxes found or regex did not match
        result = "0 0 0 0"
        #image_no_prediction += 1

    output_file_path = os.path.join("qwen/inference-results/", Path(file_name).stem + '.txt')

    print(f"Response for {file_name}: {response}. Normalized coordinates result: {result}")
    with open(output_file_path, 'w') as file:
        file.write(result)

print(f"Images with results {image_prediction}, Images without results: {image_no_prediction}")

['0.4687500000 0.3925781250 0.3554687500 0.2832031250']
['(298,257)', '(662,547)']


AttributeError: 'NoneType' object has no attribute 'seek'

In [9]:
def normalize_and_format_bounding_boxes(matches, image_width, image_height):
    normalized_boxes = []
    for i in range(0, len(matches), 2):
        x1, y1 = map(int, matches[i].strip('()').split(','))
        x2, y2 = map(int, matches[i+1].strip('()').split(','))

        center_x = (x1 + x2) / 2 / image_width
        center_y = (y1 + y2) / 2 / image_height
        width = (x2 - x1) / image_width
        height = (y2 - y1) / image_height

        normalized_boxes.append(f"{center_x:.10f} {center_y:.10f} {width:.10f} {height:.10f}")
    return normalized_boxes

# Function to draw bounding boxes
def draw_bounding_boxes(image, matches):
    draw = ImageDraw.Draw(image)
    for i in range(0, len(matches), 2):
        x1, y1 = map(int, matches[i].strip('()').split(','))
        x2, y2 = map(int, matches[i+1].strip('()').split(','))
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)

# Processing images
image_prediction = 0
image_no_prediction = 0

for file_name in images_filenames:
    image_path = file_name
    
    query = tokenizer.from_list_format([
        {'image': file_name},
        {'text': 'Give me the bounding box of drones in the photo, if any exist.'},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)

    # Use regex to find all coordinates inside parentheses
    matches = re.findall(r'\(\d+,\d+\)', response)
    
    if matches and len(matches) % 2 == 0:
        # Open the image to get its dimensions
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Normalize the coordinates and format as required
        formatted_bounding_boxes = normalize_and_format_bounding_boxes(matches, image_width, image_height)

        print(formatted_bounding_boxes)
        print(matches)

        # Draw bounding boxes
        with Image.open(image_path) as img:
            draw_bounding_boxes(img, matches)
            img_with_boxes_path = os.path.join("qwen/inference-results/", Path(file_name).stem + '.jpg')
            img.save(img_with_boxes_path)
        
        image_prediction += 1
    else:
        result = "0 0 0 0"
        image_no_prediction += 1

    output_file_path = os.path.join("qwen/inference-results/", Path(file_name).stem + '.txt')
    result = '\n'.join(formatted_bounding_boxes) if matches else result

    print(f"Response for {file_name}: {response}. Normalized coordinates result: {result}")
    with open(output_file_path, 'w') as file:
        file.write(result)

print(f"Images with results: {image_prediction}, Images without results: {image_no_prediction}")

['0.4692382812 0.3916015625 0.3564453125 0.2851562500']
['(298,255)', '(663,547)']
Response for ../../../teamspace/uploads/img1004230.jpg: <ref> drones</ref><box>(298,255),(663,547)</box> in the photo. Normalized coordinates result: 0.4692382812 0.3916015625 0.3564453125 0.2851562500
['0.4887695312 0.4780273438 0.2685546875 0.1826171875']
['(363,396)', '(638,583)']
Response for ../../../teamspace/uploads/img1004231.jpg: <ref> drones</ref><box>(363,396),(638,583)</box> in the photo. Normalized coordinates result: 0.4887695312 0.4780273438 0.2685546875 0.1826171875
['0.6181640625 0.3979492188 0.0722656250 0.0810546875']
['(596,366)', '(670,449)']
Response for ../../../teamspace/uploads/img1004232.jpg: <ref> drones in the photo</ref><box>(596,366),(670,449)</box>. Normalized coordinates result: 0.6181640625 0.3979492188 0.0722656250 0.0810546875
['0.4853515625 0.4619140625 0.7929687500 0.4882812500']
['(91,223)', '(903,723)']
Response for ../../../teamspace/uploads/img1004233.jpg: <ref> d